In [ ]:
# default_exp core

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Rotation

The bones are rotated with quaternions.

The angle between the two PC1 vectors is taken. The object is then rotated (by a quaternion) around the cross product between the PC1 vectors.

The new angles between the next PCs are calculates and the process is repeated for the other PCs

In [ ]:
#hide
# import os
# import math
# import numpy as np
# import pandas as pd
# import scipy.io
# from pathlib import Path
# from mayavi import mlab
# import quaternion as quat
# from sklearn.decomposition import PCA

In [ ]:
#export
def quaternion_rotation_from_angle(v, c_axis, theta):
    
    """
    rotates vector around axis by theta
    
    """
    
    rotation_axis = np.array([0.] + c_axis)
    axis_angle = (theta*0.5) * rotation_axis/np.linalg.norm(rotation_axis)

    vec = np.quaternion(*v)

    # quaternion from exp of axis angle
    qlog = np.quaternion(*axis_angle)
    q = np.exp(qlog)

    # double cover quaternion rotation
    v_prime = q * vec * np.conjugate(q)

    return v_prime.imag , q

In [ ]:
#export
def quaternion_rotation_from_quaternion(v, q):
    """
    rotates vector by quaternion
    
    """
    
    # double cover quaternion rotation
    vec = np.quaternion(*v)
    
    v_prime = q * vec * np.conjugate(q)
    
    return v_prime.imag

In [ ]:
#export
def rotate(bone_f1, bone_f2, interpolate = False, scale_factor= 2):

    if interpolate == True: 
        print(f'scalling bone by {scale_factor}')
        bone_f1.scale(scale_factor)
    
    # center bones too 0,0,0,
    bone_f1.center_to_origin()
    bone_f2.center_to_origin()

    # PCA on bones
    bone_f1.get_pca()
    bone_f2.get_pca()

    # for 1 to 3 principle conponents of the object
    for i in range(1, 4):

        # takes cross product axis
        cross_product_axis = np.cross(
            getattr(bone_f1, f'pc{i}'),
            getattr(bone_f2, f'pc{i}'))

        # finds angle between PCs for f1 vs f2
        theta, vector = angle(
            getattr(bone_f1, f'pc{i}'),
            getattr(bone_f2, f'pc{i}'))

        # sets any new values needed
        setattr(bone_f1, f'pc{i}', vector)

        # rotates each PC of the bone
        for n in range(1, 4):
            transformed_pc, q = quaternion_rotation_from_angle(
                v=getattr(bone_f1, f'pc{n}'),
                c_axis=cross_product_axis,
                theta=theta)
            
            # sets new PCA
            setattr(bone_f1, f'pc{n}', transformed_pc)
            
        #logs quaternion
        setattr(bone_f1,f'q_prod_{i}', q)
        
    q_total = getattr(bone_f1, 'q_prod_3') * getattr(bone_f1, 'q_prod_2') * getattr(bone_f1, 'q_prod_1')
            
            
    # rotates xyz array with the quaterion product
    rotated_xyz = np.apply_along_axis(
        quaternion_rotation_from_quaternion,1, 
        getattr(bone_f1, 'xyz'),
        q_total)
        
    setattr(bone_f1, 'xyz', rotated_xyz)

    bone_f1.reset_position()
    bone_f2.reset_position()

    # reduce bone to orginal size 
    if interpolate == True: 
        print(f'scalling bone by {1/scale_factor}')
        bone_f1.scale(1/scale_factor)   
        

    if bone_f1.dtype == 'stl':

        #update internal data
        bone_f1.data.v0 , bone_f1.data.v1, bone_f1.data.v2 = np.array_split(bone_f1.xyz, 3)
        bone_f1.data.update_normals()

In [ ]:
#hide
# from nbdev.export import notebook2script; notebook2script()